In [20]:
import numpy as np
import random
import open3d as o3d
from scipy.spatial import distance, cKDTree

# Define the basic shape classes with more detailed compatibility checks
class Plane:
    def __init__(self, point, normal):
        self.point = point
        self.normal = normal

    def is_compatible(self, point, normal, epsilon, alpha):
        distance_to_plane = np.dot(point - self.point, self.normal)
        normal_deviation = np.arccos(np.dot(normal, self.normal))
        return abs(distance_to_plane) < epsilon and normal_deviation < alpha

class Sphere:
    def __init__(self, center, radius):
        self.center = center
        self.radius = radius

    def is_compatible(self, point, normal, epsilon, alpha):
        distance_to_sphere = np.linalg.norm(point - self.center) - self.radius
        return abs(distance_to_sphere) < epsilon

class Cylinder:
    def __init__(self, axis_point, axis_direction, radius):
        self.axis_point = axis_point
        self.axis_direction = axis_direction
        self.radius = radius

    def is_compatible(self, point, normal, epsilon, alpha):
        projected_point = point - np.dot(point - self.axis_point, self.axis_direction) * self.axis_direction
        distance_to_axis = np.linalg.norm(projected_point - self.axis_point) - self.radius
        return abs(distance_to_axis) < epsilon

class Cone:
    def __init__(self, apex, axis_direction, angle):
        self.apex = apex
        self.axis_direction = axis_direction
        self.angle = angle

    def is_compatible(self, point, normal, epsilon, alpha):
        direction = point - self.apex
        projected_angle = np.arccos(np.dot(direction, self.axis_direction) / np.linalg.norm(direction))
        return abs(projected_angle - self.angle) < alpha

class Torus:
    def __init__(self, center, axis_direction, major_radius, minor_radius):
        self.center = center
        self.axis_direction = axis_direction
        self.major_radius = major_radius
        self.minor_radius = minor_radius

    def is_compatible(self, point, normal, epsilon, alpha):
        vector_from_center = point - self.center
        projection_on_axis = np.dot(vector_from_center, self.axis_direction) * self.axis_direction
        major_circle_center = self.center + projection_on_axis
        distance_to_major_circle = np.linalg.norm(vector_from_center - major_circle_center)
        return abs(distance_to_major_circle - self.minor_radius) < epsilon

# Functions to generate shape candidates
def generate_plane(p1, p2, p3):
    normal = np.cross(p2 - p1, p3 - p1)
    if np.linalg.norm(normal) != 0:
        normal /= np.linalg.norm(normal)
    else:
        normal = np.array([0, 0, 1])  # Assign a default normal if zero-length
    return Plane(point=p1, normal=normal)

def generate_sphere(p1, n1, p2, n2):
    line1 = p1 + n1
    line2 = p2 + n2
    center = (p1 + p2) / 2  # Simplified calculation
    radius = (np.linalg.norm(p1 - center) + np.linalg.norm(p2 - center)) / 2
    return Sphere(center=center, radius=radius)

def generate_cylinder(p1, n1, p2, n2):
    axis_direction = np.cross(n1, n2)
    if np.linalg.norm(axis_direction) != 0:
        axis_direction /= np.linalg.norm(axis_direction)
    else:
        axis_direction = np.array([1, 0, 0])  # Assign a default direction if zero-length
    axis_point = (p1 + p2) / 2
    radius = np.linalg.norm(np.cross(p2 - p1, axis_direction)) / np.linalg.norm(axis_direction)
    return Cylinder(axis_point=axis_point, axis_direction=axis_direction, radius=radius)

def generate_cone(p1, n1, p2, n2, p3, n3):
    apex = np.mean([p1, p2, p3], axis=0)
    axis_direction = np.mean([n1, n2, n3], axis=0)
    if np.linalg.norm(axis_direction) != 0:
        axis_direction /= np.linalg.norm(axis_direction)
    else:
        axis_direction = np.array([0, 0, 1])  # Assign a default direction if zero-length
    angle = np.mean([np.arccos(np.dot(n1, axis_direction)),
                     np.arccos(np.dot(n2, axis_direction)),
                     np.arccos(np.dot(n3, axis_direction))])
    return Cone(apex=apex, axis_direction=axis_direction, angle=angle)

def generate_torus(p1, n1, p2, n2, p3, n3, p4, n4):
    center = np.mean([p1, p2, p3, p4], axis=0)
    axis_direction = np.mean([n1, n2, n3, n4], axis=0)
    if np.linalg.norm(axis_direction) != 0:
        axis_direction /= np.linalg.norm(axis_direction)
    else:
        axis_direction = np.array([0, 1, 0])  # Assign a default direction if zero-length
    major_radius = np.linalg.norm(p1 - center)
    minor_radius = np.mean([np.linalg.norm(np.cross(p - center, axis_direction)) for p in [p1, p2, p3, p4]])
    return Torus(center=center, axis_direction=axis_direction, major_radius=major_radius, minor_radius=minor_radius)

# Example function to compute normals using nearest neighbors
def compute_normal(point_cloud, k=10):
    """
    Compute surface normals for a point cloud using the k-nearest neighbors.
    
    Args:
    - point_cloud: A NumPy array of shape (N, 3) representing the 3D points.
    - k: Number of nearest neighbors to consider for normal computation.
    
    Returns:
    - normals: A NumPy array of shape (N, 3) representing the normal vectors.
    """
    tree = cKDTree(point_cloud)
    normals = []
    for point in point_cloud:
        distances, indices = tree.query(point, k=k)
        neighbors = point_cloud[indices]
        centroid = neighbors.mean(axis=0)
        cov_matrix = np.cov(neighbors - centroid, rowvar=False)
        eigvals, eigvecs = np.linalg.eigh(cov_matrix)
        normal = eigvecs[:, np.argmin(eigvals)]
        normal /= np.linalg.norm(normal) if np.linalg.norm(normal) != 0 else 1
        normals.append(normal)
    normals = np.array(normals)
    return normals

# Main algorithm to extract shapes from point cloud P
def extract_shapes(point_cloud, normals, pt, tau, epsilon, alpha):
    extracted_shapes = []
    octree = cKDTree(point_cloud)
    candidates = []
    original_indices = np.arange(len(point_cloud))  # Track original indices

    while True:
        candidates.extend(generate_candidates(point_cloud, normals, octree, epsilon, original_indices))
        
        if not candidates:  # If no candidates were generated, break the loop
            break
        
        best_shape = best_candidate(candidates, point_cloud, normals, epsilon, alpha)

        if adaptive_stopping_condition(candidates, pt):
            extracted_shapes.append(best_shape)
            
            # Filter points that are not compatible with the best shape
            new_point_cloud = []
            new_normals = []
            new_indices = []
            for i, (p, n) in enumerate(zip(point_cloud, normals)):
                if not best_shape.is_compatible(p, n, epsilon, alpha):
                    new_point_cloud.append(p)
                    new_normals.append(n)
                    new_indices.append(original_indices[i])
            
            point_cloud = np.array(new_point_cloud).reshape(-1, 3)  # Ensure point_cloud is 2D
            normals = np.array(new_normals).reshape(-1, 3)          # Ensure normals is 2D
            original_indices = np.array(new_indices)  # Update original indices
            candidates = []

            if len(point_cloud) > 0:  # Only rebuild octree if point cloud is not empty
                octree = cKDTree(point_cloud)

        if len(point_cloud) == 0 or adaptive_stopping_condition(candidates, pt):
            break

    return extracted_shapes

def generate_candidates(point_cloud, normals, octree, epsilon, original_indices):
    candidates = []
    for _ in range(num_candidates):
        # Randomly sample a point
        index1 = random.randint(0, len(point_cloud) - 1)
        point1, normal1 = point_cloud[index1], normals[index1]

        # Use octree to find local points within a radius
        local_indices = octree.query_ball_point(point1, epsilon)
        if len(local_indices) < minimal_sample_size - 1:
            continue

        # Map local_indices to the current point cloud
        local_indices = random.sample(local_indices, minimal_sample_size - 1)
        sample_points = [point1] + [point_cloud[i] for i in local_indices]
        sample_normals = [normal1] + [normals[i] for i in local_indices]

        # Generate candidates for each shape type
        if len(sample_points) == 3:
            candidates.append(generate_plane(*sample_points))
            candidates.append(generate_cone(*sample_points, *sample_normals))
        elif len(sample_points) == 2:
            candidates.append(generate_sphere(*sample_points, *sample_normals))
            candidates.append(generate_cylinder(*sample_points, *sample_normals))
        elif len(sample_points) == 4:
            candidates.append(generate_torus(*sample_points, *sample_normals))

    return candidates

def score_function(shape, point_cloud, normals, epsilon, alpha):
    compatible_points = [
        p for p, n in zip(point_cloud, normals) if shape.is_compatible(p, n, epsilon, alpha)
    ]
    return len(compatible_points)

def best_candidate(candidates, point_cloud, normals, epsilon, alpha):
    return max(candidates, key=lambda c: score_function(c, point_cloud, normals, epsilon, alpha))

def adaptive_stopping_condition(candidates, pt):
    if not candidates:  # If candidates list is empty, return False
        return False
    
    # Evaluate the highest score among candidates to determine stopping
    best_candidate_score = max(score_function(c, point_cloud, normals, epsilon, alpha) for c in candidates)
    probability_confidence = 1 - np.exp(-best_candidate_score / len(candidates))
    return probability_confidence > pt

# Constants and parameters
num_candidates = 100
minimal_sample_size = 3  # Adjust based on shape type
epsilon = 0.01  # Maximum distance for compatibility
alpha = 0.1     # Maximum normal deviation
pt = 0.99  # Probability threshold for accepting a candidate
tau = 10  # Minimal shape size

pcd = o3d.io.read_point_cloud("filtered.pcd")
downpcd = pcd.voxel_down_sample(voxel_size=0.005)
point_cloud = np.asarray(downpcd.points)
normals = compute_normal(point_cloud) 

extracted_shapes = extract_shapes(point_cloud, normals, pt, tau, epsilon, alpha)

In [22]:
import open3d as o3d
import numpy as np
from scipy.spatial.transform import Rotation as R

def visualize_one_shape(point_cloud, shape):
    # Create an Open3D point cloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    
    geometries = [pcd]  # Start with the original point cloud

    if isinstance(shape, Sphere):
        # Create a sphere mesh
        sphere_mesh = o3d.geometry.TriangleMesh.create_sphere(radius=shape.radius)
        sphere_mesh.translate(shape.center)
        sphere_mesh.paint_uniform_color([0, 1, 0])  # Color the sphere green
        geometries.append(sphere_mesh)
    
    elif isinstance(shape, Plane):
        # Create a plane mesh
        plane_mesh = o3d.geometry.TriangleMesh.create_box(width=1, height=1, depth=0.01)
        plane_mesh.translate(shape.point)
        plane_mesh.paint_uniform_color([1, 0, 0])  # Color the plane red
        geometries.append(plane_mesh)
    
    elif isinstance(shape, Cylinder):
        # Create a cylinder mesh
        cylinder_mesh = o3d.geometry.TriangleMesh.create_cylinder(radius=shape.radius, height=1)
        cylinder_mesh.translate(shape.axis_point - shape.axis_direction * 0.5)
        cylinder_mesh.paint_uniform_color([0, 0, 1])  # Color the cylinder blue
        geometries.append(cylinder_mesh)

    # Visualize the point cloud with the extracted shape overlaid
    o3d.visualization.draw_geometries(geometries)


# Visualize the original point cloud with the extracted shapes overlaid
visualize_extracted_shapes_on_cloud(point_cloud, extracted_shapes)